### 参考：http://datawhale.club/t/topic/41

In [1]:
import numpy as np
import pandas as pd

### UserCF
- 计算user之间的相似性
- 获取目标用户的前n个相似用户
- 计算最终得分

In [2]:
# 定义数据集， 也就是那个表格， 注意这里我们采用字典存放数据， 因为实际情况中数据是非常稀疏的， 很少有情况是现在这样
def loadData():
    '''
    第一个字典是 物品-用户 的评分映射(倒排表)， 键是物品1-5， 用A-E来表示， 每一个值又是一个字典， 表示的是每个用户对该物品的打分。 
    第二个字典是 用户-物品 的评分映射， 键是上面的五个用户， 用1-5表示， 值是该用户对每个物品的打分
    '''
    items={'A': {1: 5, 2: 3, 3: 4, 4: 3, 5: 1},
           'B': {1: 3, 2: 1, 3: 3, 4: 3, 5: 5},
           'C': {1: 4, 2: 2, 3: 4, 4: 1, 5: 5},
           'D': {1: 4, 2: 3, 3: 3, 4: 5, 5: 2},
           'E': {2: 3, 3: 5, 4: 4, 5: 1}
          }
    users={1: {'A': 5, 'B': 3, 'C': 4, 'D': 4},
           2: {'A': 3, 'B': 1, 'C': 2, 'D': 3, 'E': 3},
           3: {'A': 4, 'B': 3, 'C': 4, 'D': 3, 'E': 5},
           4: {'A': 3, 'B': 3, 'C': 1, 'D': 5, 'E': 4},
           5: {'A': 1, 'B': 5, 'C': 5, 'D': 2, 'E': 1}
          }
    return items,users

items, users = loadData()
item_df = pd.DataFrame(items).T
user_df = pd.DataFrame(users).T

#### 计算用户之间的相似性

In [3]:
"""计算用户相似性矩阵"""
similarity_matrix = pd.DataFrame(np.zeros((len(users), len(users))), index=[1, 2, 3, 4, 5], columns=[1, 2, 3, 4, 5])

# 遍历每条 用户-物品 评分数据
for userID in users:
    for otheruserId in users:     # 再次遍历 用户-物品 评分数据，目的是找到不同的user的向量表示
        vec_user = []             # 记录user的vector，用于后续计算相似度
        vec_otheruser = []        # 记录user的vector，用于后续计算相似度
        if userID != otheruserId:
            for itemId in items:   # 遍历物品-用户评分数据
                itemRatings = items[itemId]        # 这也是个字典  每条数据为所有用户对当前物品的评分
                if userID in itemRatings and otheruserId in itemRatings:  # 说明两个用户都对该物品评过分
                    vec_user.append(itemRatings[userID])
                    vec_otheruser.append(itemRatings[otheruserId])
            # 这里可以获得相似性矩阵(共现矩阵)，利用两个user的向量进行计算
            similarity_matrix[userID][otheruserId] = np.corrcoef(np.array(vec_user), np.array(vec_otheruser))[0][1]
            #similarity_matrix[userID][otheruserId] = cosine_similarity(np.array(vec_user), np.array(vec_otheruser))[0][1]

In [4]:
similarity_matrix

,1,2,3,4,5
1,0.000000,0.852803,0.707107,0.000000,-0.792118
2,0.852803,0.000000,0.467707,0.489956,-0.900149
3,0.707107,0.467707,0.000000,-0.161165,-0.466569
4,0.000000,0.489956,-0.161165,0.000000,-0.641503
5,-0.792118,-0.900149,-0.466569,-0.641503,0.000000


#### 找到前n个最相似的用户

In [5]:
n = 2
similarity_users = similarity_matrix[1].sort_values(ascending=False)[:n].index.tolist()    # [2, 3]   也就是用户1和用户2

#### 计算最终得分

In [6]:
"""计算最终得分"""
base_score = np.mean(np.array([value for value in users[1].values()]))
weighted_scores = 0.
corr_values_sum = 0.
for user in similarity_users:  # [2, 3]
    corr_value = similarity_matrix[1][user]            # 两个用户之间的相似性
    mean_user_score = np.mean(np.array([value for value in users[user].values()]))    # 每个用户的打分平均值
    weighted_scores += corr_value * (users[user]['E']-mean_user_score)      # 加权分数
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores / corr_values_sum
print('用户Alice对物品5的打分: ', final_scores)
user_df.loc[1]['E'] = final_scores
user_df

用户Alice对物品5的打分:  4.871979899370592


,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.87198
2,3.0,1.0,2.0,3.0,3.00000
3,4.0,3.0,4.0,3.0,5.00000
4,3.0,3.0,1.0,5.0,4.00000
5,1.0,5.0,5.0,2.0,1.00000


#### UserCF的确缺点：
- 数据稀疏性：
        
        计算user间的相似性时，需要用到两个user对相同item的行为/评分，来建立不同user间的向量和计算相似度。大型推荐系统种，物品item数量很多，用户可能购买了其中不到1%的商品，不同用户间的购买重叠率较低，导致两个user无重叠的item，无法找到目标用户的相似用户，这使UserCF不适用于那些用户行为是低频的场景，如大型推荐系统种的购买行为。

- 算法扩展性：

        基于user的协同过滤需要维护用户相似度矩阵，以便快速找出top n相似的用户，存储开销非常大，不适用于user数据量大的情况。

### ItemCF
- 计算物品之间的相似性
- 获取最相似的前n个物品
- 计算最终得分
- 使用数据示例同UserCF

In [7]:
"""计算物品的相似度矩阵"""
similarity_matrix = pd.DataFrame(np.ones((len(items), len(items))), index=['A','B','C','D','E'], columns=['A','B','C','D','E'])

for itemID in items:
    for otheritemID in items:
        vec_item = []
        vec_otheritem = []
        if itemID != otheritemID:
            for userID in users:
                userRating = users[userID]
                if itemID in userRating and otheritemID in userRating:
                    vec_item.append(userRating[itemID])
                    vec_otheritem.append(userRating[otheritemID])
            similarity_matrix[itemID][otheritemID] = np.corrcoef(np.array(vec_item), np.array(vec_otheritem))[0][1]

In [8]:
similarity_matrix

,A,B,C,D,E
A,1.000000,-0.476731,-0.123091,0.532181,0.969458
B,-0.476731,1.000000,0.645497,-0.310087,-0.478091
C,-0.123091,0.645497,1.000000,-0.720577,-0.427618
D,0.532181,-0.310087,-0.720577,1.000000,0.581675
E,0.969458,-0.478091,-0.427618,0.581675,1.000000


#### 找到top n相似物品，计算最终得分

In [9]:
"""得到与物品5相似的前n个物品"""
n = 2
similarity_items = similarity_matrix['E'].sort_values(ascending=False)[1:n+1].index.tolist()       # ['A', 'D']

"""计算最终得分"""
base_score = np.mean(np.array([value for value in items['E'].values()]))
weighted_scores = 0.
corr_values_sum = 0.
for item in similarity_items:  # ['A', 'D']
    corr_value = similarity_matrix['E'][item]            # 两个物品之间的相似性
    mean_item_score = np.mean(np.array([value for value in items[item].values()]))    # 每个物品的打分平均值
    weighted_scores += corr_value * (users[1][item]-mean_item_score)      # 加权分数
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores / corr_values_sum
print('用户Alice对物品5的打分: ', final_scores)
user_df.loc[1]['E'] = final_scores
user_df

用户Alice对物品5的打分:  4.6


,A,B,C,D,E
1,5.0,3.0,4.0,4.0,4.6
2,3.0,1.0,2.0,3.0,3.0
3,4.0,3.0,4.0,3.0,5.0
4,3.0,3.0,1.0,5.0,4.0
5,1.0,5.0,5.0,2.0,1.0


### 协同过滤的天然缺陷：推荐系统头部效应明显， 处理稀疏向量的能力弱
协同过滤无法将两个物品相似的信息推广到其他物品的相似性上。 导致的问题是热门物品具有很强的头部效应， 容易跟大量物品产生相似， 而尾部物品由于特征向量稀疏，导致很少被推荐。特征太稀疏，会缺乏相似性计算的直接数据。